# Examine a single word  


In [ ]:
import os, random
import tensorflow as tf
import pandas as pd
import altair as alt
import numpy as np
import meta, data_wrangling, evaluate, troubleshooting
import matplotlib.pyplot as plt
from importlib import reload

In [ ]:
import modeling_useful_init as modeling


# Word options in 100 random
 - pans, loose, youth, finds, strifes
 - all others: ['wasps', 'taut', 'lived', 'close', 'side', 'mope', 'loose', 'hocks', 'join', 'shared', 'wort', 'her', 'thanks', 'lee', 'whig', 'lot', 'tie', 'walls', 'fronts', 'plops', 'kill', 'gauze', 'grief', 'block', 'grind', 'homes', 'keep', 'plume', 'fart', 'lien', 'rags', 'flanks', 'wastes', 'true', 'praised', 'cram', 'shed', 'dent', 'breadths', 'pale', 'jeeps', 'peep', 'trades', 'chin', 'read', 'rungs', 'lolled', 'strifes', 'picks', 'braille', 'wailed', 'when', 'thin', 'fit', 'slut', 'cabs', 'grave', 'lights', 'coos', 'pans', 'gripe', 'whip', 'hug', 'squads', 'chip', 'lurked', 'filths', 'theme', 'craze', 'youth', 'watt', 'squid', 'tsars', 'soups', 'string', 'moose', 'zinc', 'class', 'snips', 'sick', 'cleave', 'shone', 'vow', 'wig', 'quakes', 'scram', 'kale', 'mag', 'yawn', 'sup', 'chow', 'pinged', 'seal', 'staffs', 'wage', 'crone', 'finds', 'moats', 'gob', 'feuds']

# Parameter block

In [ ]:
# code_name = 'OP'
# code_name = 'Refrac3_local'
# code_name = 'Refrac3_after6'
# code_name = 'Refrac3_after6_nodc'
# code_name = 'Refrac3_after6_init_act0'
code_name = 'Refrac3_after6_informative'
selected_word = 'pans'
epoch = 300
testset_name = 'train_r100'

## Instantiate Diagnosis class

In [ ]:
reload(troubleshooting)
d = troubleshooting.Diagnosis(code_name)

In [ ]:
data = data_wrangling.MyData()

In [ ]:
m = modeling.MyModel(d.cfg)

In [ ]:
m.build()
m.set_active_task('triangle')

In [ ]:
m([data.np_representations['ort'][:100,:]] * 12)

# Intact SEM

In [ ]:
df_sem1 = d.subset_df('sem', 1)
plotter = troubleshooting.Plots(df_sem1)
plotter.activation()

Stonge kick from bias?

In [ ]:
bias_df

In [ ]:
bias_df = df_sem1.loc[(df_sem1.variable=='bias') & (df_sem1.target_act==1)]

alt.Chart(bias_df).mark_bar().encode(
    y='mean(value)',
    x='unit:N',
)

In [ ]:
d.eval(testset_name, task='triangle', epoch=epoch)
d.set_target_word(selected_word)
intact_sem = d.plot_one_layer('sem')
intact_sem.save(os.path.join(d.cfg.plot_folder, f"intact_sem_{selected_word}.html"))
intact_sem

# OS SEM

In [ ]:
d.eval(testset_name, task='ort_sem', epoch=epoch)
d.set_target_word(selected_word)
OS_sem = d.plot_one_layer('sem')
OS_sem.save(os.path.join(d.cfg.plot_folder, f"OS_sem_{selected_word}.html"))
OS_sem

# OPS SEM

In [ ]:
d.eval(testset_name, task='exp_ops', epoch=epoch)
d.set_target_word(selected_word)
OPS_sem = d.plot_one_layer('sem')
OPS_sem.save(os.path.join(d.cfg.plot_folder, f"OPS_sem_{selected_word}.html"))
OPS_sem

# Intact PHO

In [ ]:
d.eval(testset_name, task='triangle', epoch=epoch)
d.set_target_word(selected_word)
print(f"Output phoneme over timeticks: {d.list_output_phoneme}")
intact_pho = d.plot_one_layer('pho')
intact_pho.save(os.path.join(d.cfg.plot_folder, f"intact_pho_{selected_word}.html"))
intact_pho


# OP PHO

In [ ]:
d.sec
troubleshooting.Plots

In [ ]:
d.eval(testset_name, task='ort_pho', epoch=epoch)
d.set_target_word(selected_word)
print(f"Output phoneme over timeticks: {d.list_output_phoneme}")
OP_pho = d.plot_one_layer('pho')
OP_pho.save(os.path.join(d.cfg.plot_folder, f"OP_pho_{selected_word}.html"))
OP_pho


# OSP PHO

In [ ]:
d.eval(testset_name, task='exp_osp', epoch=epoch)
d.set_target_word(selected_word)
print(f"Output phoneme over timeticks: {d.list_output_phoneme}")
OSP_pho = d.plot_one_layer('pho')
OSP_pho.save(os.path.join(d.cfg.plot_folder, f"OSP_pho_{selected_word}.html"))
OSP_pho


# Extra

## Intactive 

In [ ]:
reload(troubleshooting)

from ipywidgets import interact


@interact(
    sel_word=d.testset_package['item'], 
    layer=['pho', 'sem'], 
    task=['triangle', 'ort_pho', 'exp_osp', 'ort_sem', 'exp_ops'], 
    epoch=(10, d.cfg.total_number_of_epoch+1, d.cfg.save_freq)
    )
def interactive_plot(sel_word, layer, task, epoch):
    d = troubleshooting.Diagnosis(code_name)
    d.eval(testset_name, task='triangle', epoch=epoch)
    d.set_target_word(sel_word)
    print(f"Output phoneme over timeticks: {d.list_output_phoneme}")
    return d.plot_one_layer('pho')

In [ ]:
reload(troubleshooting)
d = troubleshooting.Diagnosis(code_name)
d.eval(testset_name, task='triangle', epoch=epoch)

def plot_raw_input_by_target(word:str) -> alt.Chart:
    d.set_target_word(word)
    print(f"Output phoneme over timeticks: {d.list_output_phoneme}")

    df = d.subset_df(layer='pho', target_act=1)
    plotter = troubleshooting.Plots(df)
    raw_input_1 = plotter.raw_inputs()

    df = d.subset_df(layer='pho', target_act=0)
    plotter = troubleshooting.Plots(df)
    raw_input_0 = plotter.raw_inputs()

    return (raw_input_1 | raw_input_0).resolve_scale(y="shared").properties(title=word)


In [ ]:
ten_words = random.sample(d.testset_package['item'], 10)
print(ten_words)

ten_plots = alt.hconcat()
for w in ten_words:
    ten_plots &= plot_raw_input_by_target(w)

ten_plots.resolve_scale(y="shared").save(os.path.join(d.cfg.plot_folder, 'ten_raw_inputs_to_PHO.html'))

- In an intact OP model, ignoring the non informative ticks, the general pattern

## We found a very weird SS-1 having a very high input in off nodes as compared to on nodes
- This pattern is also exist in PHO layer
Consider PHO layer in a OP model

1) Tick 1 is non-informative (No information from O had propagated to P yet) --> tick 1 is fixed across words
2) All input is coming from Act P at 0 (=0.5) matmul w_pp. This fixed value is summarized in below density plot
3) Functionally it serve as the knowledge (obtain from pretraining) embedded in w_pp, reflecting some degree of direct (1-to-1) coherence coactivation in the phonological layer
4) Put in the more general context, bias, PP, and CP reflects probably(?) reflects -> independent activation likelihood in each node, immediate coherence coactivation (1-to-1), and slightly more complex (because it goes through a hidden layer) (but compressed) coherence coactivation???
5) Their relative strength is summarized in below density plot



### Tick 1 input density at SEM

In [ ]:
w_hos_hs = d.get_weight(name="w_hos_hs")
w_ss = d.get_weight(name="w_ss")
bias_s = d.get_weight(name="bias_s")
w_cs = d.get_weight(name="w_cs")

data = {
    'ss1': 0.5 * np.sum(w_ss, axis=0), # Lazy matmul. 
    'cs1': 0.5 * np.sum(w_cs, axis=0),
    'os1': 0.5 * np.sum(w_hos_hs, axis=0),
    'bias1': bias_s
}

df = pd.DataFrame.from_dict(data)

fig, ax = plt.subplots(1, 4, figsize=(15,6))
for i, k in enumerate(data.keys()):
    df[k].plot.density(ax=ax[i], title=k)
    


### Tick 1 input density at PHO

In [ ]:
w_hop_hp = d.get_weight(name="w_hop_hp")
w_pp = d.get_weight(name="w_pp")
bias_p = d.get_weight(name="bias_p")
w_cp = d.get_weight(name="w_cp")

data = {
    'pp1': 0.5 * np.sum(w_pp, axis=0), # Lazy matmul. 
    'cp1': 0.5 * np.sum(w_cp, axis=0),
    'op1': 0.5 * np.sum(w_hop_hp, axis=0),
    'bias1': bias_p
}

df = pd.DataFrame.from_dict(data)

fig, ax = plt.subplots(1, 4, figsize=(15,6))
for i, k in enumerate(data.keys()):
    df[k].plot.density(ax=ax[i], title=k)
    


# Extreme case node 1918

- PP1 is very large... compare to other input (CP1, bias1)
- All pp1, cp1, bias1 is mostly negative
- There are a small number of nodes having a strong positive value 